In [ ]:
##PS ophyd

In [ ]:
from ophyd import Component, Device, EpicsSignal, EpicsSignalRO
from ophyd.status import Status

class pwrspl(Device):
    # Señales de lectura (readback)
    voltage_RB  = Component(EpicsSignalRO, "PWRSPL:readV")
    current_RB  = Component(EpicsSignalRO, "PWRSPL:readI")

    # Señal de setpoint
    current_SP  = Component(EpicsSignal,   "PWRSPL:setI")

    # Señales de encendido/apagado
    on_cmd      = Component(EpicsSignal,   "PWRSPL:on")
    off_cmd     = Component(EpicsSignal,   "PWRSPL:off")
    # Estado de salida (asumimos que `PWRSPL:on` devuelve 1 cuando está on)
    onoff_status= Component(EpicsSignalRO, "PWRSPL:on")

    def set(self, setpoint):
        stat = Status(timeout=30.0)
        print(f"onoff stat: {self.onoff_status.get()}")
        # Asegurar que esté encendido
        if self.onoff_status.get() == "":
            print("PS is OFF. Switching ON...")
            self.on_cmd.put("1")

        # Poner el setpoint
        self.current_SP.put(setpoint)

        # Esperar hasta que el readback y el setpoint coincidan (±0.1)
        while True:
            if abs(self.current_RB.get() - self.current_SP.get()) < 0.1:
                stat.set_finished()
                break

        return stat

    def toff(self):
        stat = Status(timeout=30.0)
        print(f"onoff stat: {self.onoff_status.get()}")
        # Asegurar que esté encendido
        if self.onoff_status.get() == "1":
            print("PS is ON. Switching OFF...")
            self.on_cmd.put("")

    


In [ ]:
##PS TEST

In [ ]:
#!/usr/bin/env python3

from bluesky import RunEngine
from bluesky.plans import scan
from bluesky.callbacks import LiveTable

def callback(status):
    print("Done:", status)

# Instantiate with the new PV prefix
psu = pwrspl("", name="psu")

In [ ]:
# Turn on and set to 4 A
status = psu.set(4)
status.add_callback(callback)
status.wait()  # block until complete

In [ ]:
# Create and configure the RunEngine
RE = RunEngine()

# Subscribe a LiveTable to show the current readback
RE.subscribe(LiveTable([psu.current_RB]))

# Perform a scan moving the PSU setpoint from 0 A to 7 A in 11 steps
RE(scan([], psu, -7, 7, 15))

In [ ]:
psu.ton()